In [ ]:
import numpy as np
from numpy import newaxis
import os
import cv2

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [ ]:
path = "data/"
folders = [f for f in os.listdir(path) if os.path.isdir(path+ f)]

In [ ]:
X = []
y = []


print('started!')

for lbl in folders:

    if(lbl == 'Red') :
        y_lbl= 0
    elif(lbl == 'Yellow') :
        y_lbl= 1
    elif(lbl == 'Green'):
        y_lbl= 2
    else:
        y_lbl = 3
    files = [n for n in os.listdir(path+lbl+'/') if (n.endswith('.png')|n.endswith('.jpg'))]
    for n in files:
        img = cv2.cvtColor(cv2.imread(path+lbl+'/'+n), cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (256,192))
        X.append(img)
        y.append(y_lbl)
        

y = np.array(y)
X = np.array(X)
X_, y_ = shuffle(X,y)

X_train, X_test, y_train, y_test = train_test_split(X_, y_, test_size=0.30)
X_train, y_train = shuffle(X_train, y_train)

In [ ]:
import tensorflow as tf
from tensorflow.contrib.layers import flatten


EPOCHS = 25
BATCH_SIZE = 128

def LeNet(x):    
    
    mu = 0
    sigma = 0.1
        
    conv1_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 3, 10), mean = mu, stddev = sigma))
    conv1_b = tf.Variable(tf.zeros(10))
    conv1   = tf.nn.conv2d(x, conv1_W, strides=[1, 1, 1, 1], padding='VALID') + conv1_b
    
    conv1 = tf.nn.relu(conv1)
  
    conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
   
    conv2_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 10, 20), mean = mu, stddev = sigma))
    conv2_b = tf.Variable(tf.zeros(20))
    conv2   = tf.nn.conv2d(conv1, conv2_W, strides=[1, 1, 1, 1], padding='VALID') + conv2_b
      
    conv2 = tf.nn.relu(conv2)
    
    conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
    
    fc0   = flatten(conv2)
        
    fc1_W = tf.Variable(tf.truncated_normal(shape=(54900, 120), mean = mu, stddev = sigma))
    fc1_b = tf.Variable(tf.zeros(120))
    fc1   = tf.matmul(fc0, fc1_W) + fc1_b
       
    fc1    = tf.nn.relu(fc1)
    
    fc2_W  = tf.Variable(tf.truncated_normal(shape=(120, 84), mean = mu, stddev = sigma))
    fc2_b  = tf.Variable(tf.zeros(84))
    fc2    = tf.matmul(fc1, fc2_W) + fc2_b
        
    fc2    = tf.nn.relu(fc2)
    
    fc3_W  = tf.Variable(tf.truncated_normal(shape=(84, 4), mean = mu, stddev = sigma))
    fc3_b  = tf.Variable(tf.zeros(4))
    logits = tf.matmul(fc2, fc3_W) + fc3_b
    
    return logits

In [ ]:
x = tf.placeholder(tf.float32, (None, 192,256,3))
y = tf.placeholder(tf.int32, (None))
one_hot_y = tf.one_hot(y, 4)

In [ ]:
rate = 0.0007

logits = LeNet(x)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)
loss_operation = tf.reduce_mean(cross_entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

In [ ]:
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver = tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train)
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train, y_train = shuffle(X_train, y_train)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y})
            
        validation_accuracy = evaluate(X_test, y_test)
        training_accuracy = evaluate(X_train,y_train)
        print("EPOCH {} ...".format(i+1))
        print("Validation Accuracy = {:.3f}".format(validation_accuracy))
        print("Training Accuracy = {:.3f}".format(training_accuracy))
        print()
        
    saver.save(sess, './lenet_cpstn')
    print("Model saved")